In [ ]:
%load_ext autoreload
%autoreload 2
import HammingCode
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import erfc
import NeuralNetworkCorrection as network
import Autoencoder
import Utils

In [20]:
# tamanho da amostra
tamanho = 1e8
variancia = 1

# amostra
tamanho_amostra = 10**3
Eb_dB_values = np.arange(-2, 11, 1)

# ML data

# Gerar dados de treinamento
word_length = 4 # Tamanho padrão da entrada do hamming
training_sample_length = 10 **3
epoches = 50
training_noisy_data = []
training_original_data = []
batch_size = 32

training_data = {
    'original': [],
    'noisy': [],
    'info': []
}
training_information = network.get_training_data_set(training_sample_length, variancia)

for eb_db in Eb_dB_values:
    info = network.generate_data_for_training(training_information, eb_db)
    training_data['info'].append(info)
    training_data['noisy'].extend(info['split_encoded_data'])
    training_data['original'].extend(info['split_original_data'])


3250.0
3250
3250


In [22]:
# Criar e treinar a rede neural
#model = network.train_neural_network(training_data, epoches, batch_size)
autoencoder = Autoencoder.create_and_train_auto_encoder(training_data, epoches, batch_size)

ValueError: Data cardinality is ambiguous:
  x sizes: 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7

In [ ]:
# Gerar dados de teste
amostras_ruidosas = []
amostras_originais = []
amostras_decodificadas = []
amostras_decodificadas_tradicional = []
amostras_decodificadas_autoencoder = []
for eb_db in Eb_dB_values:
    ruidosa, original = network.generate_data_for_training(1, tamanho_amostra, eb_db)
    amostras_originais.append(original)
    amostras_ruidosas.append(ruidosa)


In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[0], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[0])))

amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[0]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[1], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[1])))

amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[1]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[2], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[2])))
amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[2]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[3], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[3])))
amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[3]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[4], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[4])))

amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[4]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[5], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[5])))

amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[5]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[6], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[6])))

amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[6]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[7], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[7])))

amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[7]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[8], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[8])))

amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[9]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[9], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[9])))

amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[9]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[10], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[10])))

amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[10]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[11], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[11])))

amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[11]))

In [ ]:
amostras_decodificadas.append(network.decode_and_correct(amostras_ruidosas[12], model))
amostras_decodificadas_tradicional.append(HammingCode.decode_sample((amostras_ruidosas[12])))

amostras_decodificadas_autoencoder.append(Autoencoder.decode(autoencoder, amostras_ruidosas[12]))

In [ ]:
BER_tradicional = []
BER_autoencoder = []
BER_ML = []

for i in range(0, len(Eb_dB_values)):
    BER_ML.append(np.sum(np.array(amostras_decodificadas[i]).astype(int) != amostras_originais[i]) / len(amostras_originais[i]))
    BER_tradicional.append(np.sum(np.array(amostras_decodificadas_tradicional[i]).astype(int) != amostras_originais[i]) / len(amostras_originais[i]))
    BER_autoencoder.append(np.sum(np.array(amostras_decodificadas_autoencoder[i]).astype(int) != amostras_originais[i]) / len(amostras_originais[i]))

# Calcular a BER teórica
Eb_values = 10**(Eb_dB_values / 10)
BER_teorica = 0.5 * erfc(np.sqrt(Eb_values))

In [ ]:
# Plotar as taxas de erro de bit simuladas e teóricas
plt.figure(figsize=(10, 6))
plt.semilogy(Eb_dB_values, BER_tradicional, linestyle = '--', label='BER Tradicional')
plt.semilogy(Eb_dB_values, BER_autoencoder, linestyle = '--', label='BER Autoencoder')
plt.semilogy(Eb_dB_values, BER_ML, linestyle = '--', label='BER ML')
plt.semilogy(Eb_dB_values, BER_teorica, linestyle='--', label='BER Teórica')
plt.xlabel('Eb/No (dB)')
plt.ylabel('BER')
plt.grid(True, which='both')
plt.legend()
plt.title('BER x Eb/No')
plt.show()